# Twitter Dog Analysis 
                         - By Abhik Sarkar

## Three Steps of Data Wrangling 
- Gathering Data
- Assessing data
- Cleaning data


Detect and document at least eight (8) quality issues and two (2) tidiness issues 

## Gathering the Data 

In [8]:
#necessary imports 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
import json
import csv
import sys
import os
import time
import requests


from tweepy import OAuthHandler


### First Part 
Loading the data from the csv file

In [6]:

df_tweet = pd.read_csv("twitter-archive-enhanced.csv")
df_tweet.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


### Second Part
Downloading the data from the internet and loading it using the pandas 

In [10]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open('image_predictions.tsv', 'wb') as file:
    file.write(response.content)``````````

Opening the tsv file

In [15]:
#open tsv file
df_images = pd.read_table('image_predictions/image-predictions.tsv',sep='\t')

In [16]:
df_images.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### Third Part
Additional Data via the Twitter API

In [ ]:
# authentication pieces
consumer_key = ""
consumer_secret = ""
access_token = ""
access_secret = ""


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth_handler=auth, 
                 wait_on_rate_limit=True, 
                 wait_on_rate_limit_notify=True)



Twitter Query using tweet_id information from the df.

In [ ]:
tweet_ids = list(df.tweet_id)

tweet_data = {}
for _id in tweet_ids:
    try:
        tweet_status = api.get_status(_id,
                                      wait_on_rate_limit=True, 
                                      wait_on_rate_limit_notify=True)
        tweet_data[str(_id)] = tweet_status._json
    except: 
        print("Error for: " + str(_id))

In [ ]:
with open('tweet_json.txt', 'w') as file:
     file.write(json.dumps(tweet_data))

In [21]:
tweet_json = open('tweet_json.txt', "r")
tweet_line =[]
tweets = tweet_json.read() 
for line in tweet_json:
    try:
        tweet = json.loads(line)
        tweet_line.append(tweet)
    except:
        continue
        
tweet_json.close()

 Converting the twitter api to pandas DataFrame to Combine with the Existing DataFrame 

In [22]:
obj = json.loads(tweets)

In [23]:
print(obj['892420643555336193'][ 'text'], obj['892420643555336193']['retweet_count'] ,obj['892420643555336193']['favorite_count'])

This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU 8315 38018


In [24]:
data = json.loads(tweets)
tweet_id,text,retweet_count,favorite_count = [],[],[],[]
for key in list(data.keys()):
    tweet_id.append(key)
    retweet_count.append(data[key]['retweet_count'])
    favorite_count.append(data[key]['favorite_count'])
df_api = pd.DataFrame([tweet_id,retweet_count,favorite_count]).T
df_api.columns = ["tweet_id","retweet_count","favorite_count"]
df_api.head(1)

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8315,38018


 #### Complete API Data

In [28]:
data["892420643555336193"]

{'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
 'id': 892420643555336193,
 'id_str': '892420643555336193',
 'text': "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 892420639486877696,
    'id_str': '892420639486877696',
    'indices': [86, 109],
    'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'url': 'https://t.co/MgUWQ76dJU',
    'display_url': 'pic.twitter.com/MgUWQ76dJU',
    'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 540, 'h': 528, 'resize': 'fit'},
     'small': {'w': 540, 'h': 528, 'resize': 'fit'},
     'large': {'w': 540, 'h': 528, 'resize': 'fit'}}}]},


## Assessing data 

#### df_tweet

In [30]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [31]:
df_tweet.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [35]:
df_tweet.isna().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [36]:
len(df_tweet)

2356

In [52]:
df_tweet[df_tweet["timestamp"] > '2017-08-02 00:00:00 +0000']

# No Tweet is beyond 1st Aug 2017


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [49]:
df_tweet.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [72]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_numerator == 0, 'text']]

text
When you're so blinded by your systematic plagiarism that you forget what day it is. 0/10 https://t.co/YbEJPkg4Ag          NaN
PUPDATE: can't see any. Even if I could, I couldn't reach them to pet. 0/10 much disappointment https://t.co/c7WXaB2nqX    NaN
Name: text, dtype: object

In [73]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_numerator == 3, 'text']]

text
This is Wesley. He's clearly trespassing. Seems rather h*ckin violent too. Weaponized forehead. 3/10 wouldn't let in https://t.co/pL7wbMRW7M    NaN
This is Alexanderson. He's got a weird ass birth mark. Dreadful at fetch. Won't eat kibble. 3/10 wtf @Target https://t.co/FmxOpf2Sgl            NaN
What hooligan sent in pictures w/out a dog in them? Churlish af. 3/10 just bc that's a neat fluffy bean bag chair https://t.co/wcwoGOkZvz       NaN
This is Elliot. He's blocking the roadway. Downright rude as hell. Doesn't care that you're already late. 3/10 https://t.co/FMUxir5pYu          NaN
This is Charl. He's a bully. Chucks that dumbbell around like its nothing. Sharp neck. Exceptionally unfluffy. 3/10 https://t.co/VfLoDZecJ7     NaN
Meet Phil. He's big af. Currently destroying this nice family home. Completely uncalled for. 3/10 not a good pupper https://t.co/fShNNhBWYx     NaN
This is Hammond. He's a peculiar pup. Loves long walks. Bark barely audible. Too many legs. 3/10 must be ra

In [50]:
df_tweet.rating_denominator.value_counts()
# Mostly it's out of 10. There are few different ones as well.

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [53]:
df_tweet.loc[df_tweet.rating_denominator == 11, 'text']

784     RT @dog_rates: After so many requests, this is...
1068    After so many requests, this is Bretagne. She ...
1662    This is Darrel. He just robbed a 7/11 and is i...
Name: text, dtype: object

In [54]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator == 11, 'text']]

text
RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…    NaN
After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ    NaN
This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5     NaN
Name: text, dtype: object

In [55]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator == 50, 'text']]

text
This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq                                 NaN
From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK    NaN
Here is a whole flock of puppers.  60/50 I'll take the lot https://t.co/9dpcw6MdWa                                                                    NaN
Name: text, dtype: object

In [56]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator == 80, 'text']]

text
Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12    NaN
Here we have an entire platoon of puppers. Total score: 88/80 would pet all at once https://t.co/y93p6FLvVw     NaN
Name: text, dtype: object

In [57]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator == 20, 'text']]

text
Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a                                                                            NaN
Yes I do realize a rating of 4/20 would've been fitting. However, it would be unjust to give these cooperative pups that low of a rating    NaN
Name: text, dtype: object

In [58]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator == 2, 'text']]

text
This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv    NaN
Name: text, dtype: object

In [59]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator == 16, 'text']]

text
I'm aware that I could've said 20/16, but here at WeRateDogs we are very professional. An inconsistent rating scale is simply irresponsible    NaN
Name: text, dtype: object

In [60]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator == 40, 'text']]

text
Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ    NaN
Name: text, dtype: object

In [61]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator == 70, 'text']]

text
The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd    NaN
Name: text, dtype: object

In [62]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator == 15, 'text']]

text
@docmisterio account started on 11/15/15    NaN
Name: text, dtype: object

In [63]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator == 90, 'text']]

text
Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1    NaN
Name: text, dtype: object

In [64]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator ==110   , 'text']]

text
Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55    NaN
Name: text, dtype: object

In [65]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator ==120   , 'text']]

text
IT'S PUPPERGEDDON. Total of 144/120 ...I think https://t.co/ZanVtAtvIq    NaN
Name: text, dtype: object

In [66]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator ==130   , 'text']]

text
Two sneaky puppers were not initially seen, moving the rating to 143/130. Please forgive us. Thank you https://t.co/kRK51Y5ac3    NaN
Name: text, dtype: object

In [67]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator ==150   , 'text']]

text
Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE    NaN
Name: text, dtype: object

In [68]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator ==170   , 'text']]

text
Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv    NaN
Name: text, dtype: object

In [69]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator ==7   , 'text']]

text
Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx    NaN
Name: text, dtype: object

In [70]:
df_tweet['text'][df_tweet.loc[df_tweet.rating_denominator ==0   , 'text']]

text
@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho    NaN
Name: text, dtype: object

#### df_images

In [39]:
df_images.isna().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

#### df_api

In [29]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 3 columns):
tweet_id          2340 non-null object
retweet_count     2340 non-null object
favorite_count    2340 non-null object
dtypes: object(3)
memory usage: 54.9+ KB


In [41]:
df_api.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8315,38018
1,892177421306343426,6142,32639
2,891815181378084864,4067,24561
3,891689557279858688,8451,41368
4,891327558926688256,9157,39551


In [40]:
df_api.isna().sum()

tweet_id          0
retweet_count     0
favorite_count    0
dtype: int64

## Cleaning the data 

### Checking DataTypes

In [43]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [44]:
# timestamp 
df_tweet["timestamp"] = pd.to_datetime(df_tweet["timestamp"])
df_tweet["retweeted_status_timestamp"]=  pd.to_datetime(df_tweet["retweeted_status_timestamp"])

In [46]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 3 columns):
tweet_id          2340 non-null object
retweet_count     2340 non-null object
favorite_count    2340 non-null object
dtypes: object(3)
memory usage: 54.9+ KB


In [47]:
df_api["tweet_id"] = df_api.tweet_id.astype(np.uint64)
df_api["retweet_count"] = df_api.retweet_count.astype(np.uint64)
df_api["favorite_count"] = df_api.favorite_count.astype(np.uint64)